In [2]:
%pip install -r ../requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 27.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.8/384.8 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: tavily-python
    Found existing installation: tavily-python 0.3.7
    Uninstalling tavily-python-0.3.7:
      Successfully uninstalled tavily-python-0.3.7
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.29
    Uninstalling langchain-core-0.2.29:
      Successfully uninstalled langchain-core-0.2.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.12
    Uninstalling langchain-0.2.12:
      Successfully uninstalled langchain-0.2.12
Note: you may need to restart the kernel to use updated packages.


Load Env Variables and Secrets

In [1]:
import os
from dotenv import load_dotenv
load_dotenv('../../../azure.env')
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-06-01"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = 'gpt-4o-mini'

Import packages

In [8]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda,RunnableParallel

Initialize the Model

In [3]:
model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

Define the Prompt Templates

In [7]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert product reviewer"),
        ("human","List all the features of the product {product_name}.")
    ]
)

# pro analysis function
def analysis_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system","You are an expert product reviewer"),
            ("human","Given these features: {features}, list the top 5 pros features")
        ]
    )
    return pros_template.format_prompt(features=features)

# cons analysis function
def analysis_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system","You are an expert product reviewer"),
            ("human","Given these features: {features}, list the top 5 cons features")
        ]
    )
    return cons_template.format_prompt(features=features)

# combine pros and cons
def combine_pros_cons(pros,cons):
    return f'Pros:\n{pros}\n\nCons:\n{cons}'

Chains

In [10]:
pros_chain = (
    RunnableLambda(lambda x:analysis_pros(x)) | model | StrOutputParser()
)

cons_chain = (
    RunnableLambda(lambda x:analysis_cons(x)) | model | StrOutputParser()
)

# final chain

chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={'pros':pros_chain,'cons':cons_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x['branches']['pros'],x['branches']['cons']))
)
result = chain.invoke({'product_name':"Mackbook Pro"})
print(result)

Pros:
Based on the features provided for the latest MacBook Pro models, here are the top five pro features:

1. **Apple Silicon Performance**:
   - The custom Apple Silicon chips (M1, M2, etc.) offer exceptional performance and efficiency, making tasks like video editing, software development, and graphic design seamless and responsive.

2. **Retina Display with True Tone**:
   - The high-resolution Retina Display enhances visuals with True Tone technology and wide color (P3) support, making it ideal for creative professionals who require accurate color representation.

3. **Long Battery Life**:
   - With up to 20 hours of web browsing or video playback, the MacBook Pro provides impressive battery longevity, allowing users to work for extended periods without needing to recharge.

4. **Thunderbolt/USB 4 Ports**:
   - The inclusion of multiple Thunderbolt/USB 4 ports facilitates high-speed data transfer and a wide range of connectivity options, making it easier to connect peripherals an